In [1]:
#这个案例跑出来的结果和配套代码中给出的结果不一样，具体原因还没找到。


import sys,random,math
from collections import Counter
import numpy as np

np.random.seed(1)
random.seed(1)
f = open('F:\DL\IMDB\\reviews.txt')
raw_reviews = f.readlines()
f.close()

tokens = list(map(lambda x:(x.split(" ")),raw_reviews))
wordcnt = Counter()
for sent in tokens:
    for word in sent:
        wordcnt[word] -= 1
vocab = list(set(map(lambda x:x[0],wordcnt.most_common())))

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

concatenated = list()
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
            concatenated.append(word2index[word])
        except:
            ""
    input_dataset.append(sent_indices)
concatenated = np.array(concatenated)
random.shuffle(input_dataset)


alpha, iterations = (0.05, 2)
hidden_size,window,negative = (50,2,5)

weights_0_1 = (np.random.rand(len(vocab),hidden_size) - 0.5) * 0.2
weights_1_2 = np.random.rand(len(vocab),hidden_size)*0

layer_2_target = np.zeros(negative+1)
layer_2_target[0] = 1

def analogy(positive=['terrible','good'],negative=['bad']):
    
    norms = np.sum(weights_0_1 * weights_0_1,axis=1)
    norms.resize(norms.shape[0],1)
    
    normed_weights = weights_0_1 * norms
    
    query_vect = np.zeros(len(weights_0_1[0]))
    for word in positive:
        query_vect += normed_weights[word2index[word]]
    for word in negative:
        query_vect -= normed_weights[word2index[word]]
    
    scores = Counter()
    for word,index in word2index.items():
        raw_difference = weights_0_1[index] - query_vect
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))
        
    return scores.most_common(10)[1:]


def sigmoid(x):
    return 1/(1 + np.exp(-x))

for rev_i,review in enumerate(input_dataset * iterations):
  for target_i in range(len(review)):
        
    # since it's really expensive to predict every vocabulary
    # we're only going to predict a random subset（每次只预测一个随机子集，因为对每个词汇都进行预测的代价过于高昂）
    target_samples = [review[target_i]]+list(concatenated\
    [(np.random.rand(negative)*len(concatenated)).astype('int').tolist()])

    left_context = review[max(0,target_i-window):target_i]
    right_context = review[target_i+1:min(len(review),target_i+window)]

    layer_1 = np.mean(weights_0_1[left_context+right_context],axis=0)
    layer_2 = sigmoid(layer_1.dot(weights_1_2[target_samples].T))
    layer_2_delta = layer_2 - layer_2_target
    layer_1_delta = layer_2_delta.dot(weights_1_2[target_samples])

    weights_0_1[left_context+right_context] -= layer_1_delta * alpha
    weights_1_2[target_samples] -= np.outer(layer_2_delta,layer_1)*alpha

  if(rev_i % 250 == 0):
    sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)
        *iterations)) + "   " + str(analogy(['elizabeth','he'],['she'])))#这里预测不同输入要进行对应的修改（如果预测analogy(['elizabeth','he'],['she'])，需要将print(analogy(['terrible','good'],['bad']))--->analogy(['elizabeth','he'],['she'])）
  sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)
        *iterations)))


#print(analogy(['terrible','good'],['bad']))



print(analogy(['elizabeth','he'],['she']))

Progress:0.99998 [('tom', -180.6080691055368), ('he', -180.62124122426113), ('christopher', -180.6743144159594), ('mr', -180.6857981744892), ('gary', -180.69961308043472), ('peter', -180.80495228533738), ('alan', -180.8087218914704), ('mrs', -180.82751406664178), ('william', -180.83052692747387)]]2696)])]9644)]])]yweird', -0.3150575345441673)][('christopher', -196.48212630761157), ('tom', -196.55550092680483), ('peter', -196.70862778324366), ('david', -196.73931317035604), ('fred', -196.80124755381456), ('mr', -196.91202334189433), ('john', -196.92023563983406), ('this', -196.94730518343948), ('de', -196.97432887501645)]
